In [ ]:
### Import

In [ ]:
import pandas as pd
import re
import numpy as np


### 1. Caricamento del dataset

In [ ]:
file_path = "./data/monsters.csv"
try:
    df = pd.read_csv(file_path)
    print(f"✅ Dataset caricato: {file_path}")
except FileNotFoundError:
    # Fallback in caso la cartella ./data non esista o il file sia nella root
    file_path = "monsters.csv"
    df = pd.read_csv(file_path)
    print(f"✅ Dataset caricato (root): {file_path}")


### 2. Elaborazione SENSES (Metrico & In-Place)
Estrae Passive Perception e Visioni (convertite in metri).
Mantiene l'ordine delle colonne originale.

In [ ]:
if 'senses' in df.columns:
    # 1. Recupera l'indice della colonna 'senses' per mantenere l'ordine
    idx = df.columns.get_loc('senses')
    
    # 2. Pre-processing stringa (gestione sicura dei NaN)
    senses_str = df['senses'].fillna('').astype(str)

    # 3. Estrazione Passive Perception (Inserimento a idx + 1)
    # Regex: Cerca 'passive Perception' seguito da cifre
    pp_extracted = senses_str.str.extract(r'passive Perception\s+(\d+)', flags=re.IGNORECASE, expand=False)
    pp_vals = pd.to_numeric(pp_extracted).fillna(0).astype(int)
    
    # Check per evitare duplicati se la cella viene rieseguita
    if 'passive_perception' not in df.columns:
        df.insert(idx + 1, 'passive_perception', pp_vals)

    # 4. Estrazione Visioni (Inserimento sequenziale dopo PP)
    visions = ['darkvision', 'blindsight', 'tremorsense', 'truesight']
    current_offset = 2 # Offset iniziale (dopo senses e passive_perception)

    for vision in visions:
        col_name = f'sense_{vision}_m'
        
        # Regex: Cerca il nome della visione seguito da cifre (feet)
        extracted = senses_str.str.extract(rf'{vision}\s+(\d+)', flags=re.IGNORECASE, expand=False)
        
        # Conversione: Feet -> Metri (x 0.3)
        vals_m = pd.to_numeric(extracted).mul(0.3).round(1).fillna(0.0)
        
        if col_name not in df.columns:
            df.insert(idx + current_offset, col_name, vals_m)
            current_offset += 1

    # 5. Rimozione colonna originale
    df = df.drop(columns=['senses'])
    print("✅ Senses elaborato: Sostituito con Passive Perception e Visioni (m).")

else:
    print("⚠️ Colonna 'senses' non trovata (già elaborata?).")


### 3. Elaborazione LANGUAGES (Metrico & In-Place)
Estrae Telepatia (metri) e Conteggio linguaggi.

In [ ]:
if 'languages' in df.columns:
    # 1. Recupera l'indice
    idx = df.columns.get_loc('languages')
    
    # 2. Pre-processing
    langs_str = df['languages'].fillna('').astype(str)
    
    # 3. Estrazione Telepatia (Inserimento a idx + 1)
    tele_extracted = langs_str.str.extract(r'telepathy\s+(\d+)', flags=re.IGNORECASE, expand=False)
    tele_vals = pd.to_numeric(tele_extracted).mul(0.3).round(1).fillna(0.0)
    
    if 'lang_telepathy_m' not in df.columns:
        df.insert(idx + 1, 'lang_telepathy_m', tele_vals)
    
    # 4. Conteggio Linguaggi (Inserimento a idx + 2)
    # Normalizza stringa (minuscolo, pulizia separatori 'and' e ';')
    normalized = langs_str.str.lower().str.replace(' and ', ',', regex=False).str.replace(';', ',', regex=False)
    # Conta elementi non vuoti
    count_vals = normalized.str.split(',').apply(lambda x: len([s for s in x if s.strip()])).astype(int)
    
    if 'languages_count' not in df.columns:
        df.insert(idx + 2, 'languages_count', count_vals)

    # 5. Rimozione colonna originale
    df = df.drop(columns=['languages'])
    print("✅ Languages elaborato: Sostituito con Telepatia (m) e Conteggio.")

else:
    print("⚠️ Colonna 'languages' non trovata (già elaborata?).")


### 4. Salvataggio

In [ ]:
try:
    df.to_csv(file_path, index=False)
    print(f"\n💾 File salvato con successo: {file_path}")
    
    # Preview colonne create
    cols_check = ['name', 'passive_perception', 'languages_count', 'lang_telepathy_m', 'sense_darkvision_m']
    print(df[[c for c in cols_check if c in df.columns]].head())
    
except PermissionError:
    print(f"\n❌ ERRORE: Il file '{file_path}' è aperto. Chiudilo e riesegui.")
